In [64]:
import boto3
from dotenv import load_dotenv
from datetime import datetime
import os
import pandas as pd

In [65]:
load_dotenv()

CSV_FILE_NAME_GENERATED_TODAY = f"rates-{datetime.today().date()}.csv".replace("-", "_")
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION_NAME = os.getenv('AWS_REGION_NAME')
AWS_S3_BUCKET_NAME = os.getenv('AWS_S3_BUCKET_NAME')

In [66]:
s3_client: boto3.client = boto3.client('s3',
                                           aws_access_key_id=AWS_ACCESS_KEY_ID,
                                           aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                                           region_name=AWS_REGION_NAME)

In [67]:
s3_client.list_objects(Bucket=AWS_S3_BUCKET_NAME)['Contents']

[{'Key': 'data.csv',
  'LastModified': datetime.datetime(2023, 8, 14, 5, 44, 34, tzinfo=tzutc()),
  'ETag': '"351a21d1eec7753673af2996aaa72d55"',
  'Size': 5827,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'da61778a50b21a24e4edce8c6997bb75c1652198d471d19f2cacb180ec255eb3'}},
 {'Key': 'rates_2023_08_13.csv',
  'LastModified': datetime.datetime(2023, 8, 13, 19, 57, 20, tzinfo=tzutc()),
  'ETag': '"7d21a50dd7260d530e7b8ba57672d81e"',
  'Size': 5822,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'da61778a50b21a24e4edce8c6997bb75c1652198d471d19f2cacb180ec255eb3'}},
 {'Key': 'rates_2023_08_14.csv',
  'LastModified': datetime.datetime(2023, 8, 14, 5, 48, 41, tzinfo=tzutc()),
  'ETag': '"c81948d3bd1eef351cbc4ec484cdc834"',
  'Size': 5822,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'da61778a50b21a24e4edce8c6997bb75c1652198d471d19f2cacb180ec255eb3'}}]

In [68]:
import io
def get_file(file_name: str = CSV_FILE_NAME_GENERATED_TODAY) -> io.StringIO:
    try:
        print(file_name)
        response = s3_client.get_object(Bucket=AWS_S3_BUCKET_NAME, Key=file_name)
        csv_content = response['Body'].read().decode('utf-8')
        return io.StringIO(csv_content)
    except Exception as e:
        print(e)
        return None


In [69]:
column_name: str = f"{datetime.today().date()}"
rates_data:io.StringIO = get_file("data.csv")
rate_today:io.StringIO = get_file()

data.csv
rates_2023_08_14.csv


In [70]:
rates_data_df:pd.DataFrame = None
rate_today_df:pd.DataFrame = pd.read_csv(rate_today,index_col=0, header=0)

In [71]:
if rates_data == None:
    rates_data_df = pd.DataFrame(rate_today_df.rename(columns={'rates': column_name}))

In [72]:
rates_data_df = pd.read_csv(rates_data, index_col=0, header=0)
rates_data_df

,symbols,countries,2023-08-13
0,AED,United Arab Emirates Dirham,4.027006
1,AFN,Afghan Afghani,92.483124
2,ALL,Albanian Lek,103.792284
3,AMD,Armenian Dram,426.368302
4,ANG,Netherlands Antillean Guilder,1.981153
...,...,...,...
165,YER,Yemeni Rial,274.475398
166,ZAR,South African Rand,20.754214
167,ZMK,Zambian Kwacha (pre-2013),9868.627272
168,ZMW,Zambian Kwacha,20.963441


In [73]:
rate_today_df

,symbols,countries,rates
0,AED,United Arab Emirates Dirham,4.018405
1,AFN,Afghan Afghani,92.519070
2,ALL,Albanian Lek,103.832625
3,AMD,Armenian Dram,426.537921
4,ANG,Netherlands Antillean Guilder,1.981923
...,...,...,...
165,YER,Yemeni Rial,273.894435
166,ZAR,South African Rand,20.802263
167,ZMK,Zambian Kwacha (pre-2013),9847.707568
168,ZMW,Zambian Kwacha,20.971877


In [74]:
rates_data_df[column_name] = rate_today_df['rates']

In [75]:
rates_data_df

,symbols,countries,2023-08-13,2023-08-14
0,AED,United Arab Emirates Dirham,4.027006,4.018405
1,AFN,Afghan Afghani,92.483124,92.519070
2,ALL,Albanian Lek,103.792284,103.832625
3,AMD,Armenian Dram,426.368302,426.537921
4,ANG,Netherlands Antillean Guilder,1.981153,1.981923
...,...,...,...,...
165,YER,Yemeni Rial,274.475398,273.894435
166,ZAR,South African Rand,20.754214,20.802263
167,ZMK,Zambian Kwacha (pre-2013),9868.627272,9847.707568
168,ZMW,Zambian Kwacha,20.963441,20.971877


In [76]:
def upload_file_to_s3(file_buffer: io.StringIO, key: str):
    try:
        s3_client.put_object(
            Bucket=AWS_S3_BUCKET_NAME, Key=key, Body=file_buffer.getvalue()
        )
    except Exception as e:
        print(e)

In [77]:
def load_data(dataframe: pd.DataFrame, key= CSV_FILE_NAME_GENERATED_TODAY):
    with io.StringIO() as csv_buffer:
        dataframe.to_csv(csv_buffer, index=True)
        upload_file_to_s3(csv_buffer, key)

In [78]:
load_data(rates_data_df, "data.csv")